# Neutron Star Class SPH Code

In [35]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import gamma
import time
import math
import matplotlib as mp
import scipy as sp
import pylab as py
import os
import matplotlib.animation as animation

plt.rcParams['axes.grid'] = True
plt.style.use('dark_background')

G = 6.67259e-11  # (km**3/kg/s**2)
#Ms = 1.1157467e60      # Mass of the sun in kg

In [36]:
from ipynb.fs.full.Neutron_Star_SPH_sph import W, gradW, getPairwiseSeparations, getDensity
from ipynb.fs.full.Neutron_Star_SPH_eos import *
from ipynb.fs.full.Neutron_Star_SPH_NS import *
from ipynb.fs.full.NS_dynamics import *

In [37]:
myEOS = EOS(name='standard') # create an equation of state object

myNS = NS("my Neutron star 1", myEOS,1,0.5)
myNS2 = NS("my Neutron star 2", myEOS,0.5,1) # double the mass and radius of the first star

NameError: name 'eos' is not defined

## odeint integration

In [ ]:
def two_body_eqm(_y, _t, _G, _m1, _m2):
    """
    differential equations of motion describing the two-body-problem
    _t: discrete time step value
    _y: state vector
    """
    # magnitude of position vector from r1 to r2
    r_mag = np.linalg.norm(_y[3:6] - _y[:3])
    c0 = _y[6:12]
    c1 = _G * _m2 * ((_y[3:6] - _y[:3]) / np.power(r_mag, 3))
    c2 = _G * _m1 * ((_y[:3] - _y[3:6]) / np.power(r_mag, 3))
    return np.concatenate((c0, c1, c2))

# ==============================================================
# simulation harness


# time array
time = np.arange(0, 480, 0.5)

# body m1 initial conditions
m1 = 1e26  # mass (kg)
r10 = np.array([0, 0, 0])  # initial position (km)
v10 = np.array([10, 20, 30])  # initial velocity (km/s)

# body m2 initial conditions
m2 = 1e26  # mass (kg)
r20 = np.array([3000, 0, 0])  # initial position (km)
v20 = np.array([0, 40, 0])  # initial velocity (km/s)

# [X1 (0), Y1 (1), Z1 (2), X2 (3), Y2 (4), Z2 (5), VX1 (6), VY1 (7), VZ1 (8), VX2 (9), VY2 (10), VZ2 (11)]
y0 = np.concatenate((r10, r20, v10, v20))

# ==============================================================
# propagate state

# simulation results
state_history = []

y = odeint(two_body_eqm, y0, time, args=(G, m1, m2))
x1,x2,y1,y2,z1,z2,vx1,vx2,vy1,vy2,vz1,vz2 = leapfrogintegrate(0,0, r10, r20, v10, v20, m1, m2)

for yk in y:
    # extract inertial positions of body 1 and body 2
    r1 = yk[:3]
    r2 = yk[3:6]

    # determine position of centre of mass
    rg = ((m1 * r1) + (m2 * r2)) / (m1 + m2)

    # position vector from m1 to m2
    r12 = r2 - r1

    # position vector from m1 to g
    r1g = rg - r1

    # position vector from g to m1
    rg1 = r1 - rg

    # position vector from g to m2
    rg2 = r2 - rg

    # save state history (yk = 0-11, rg = 12-14, r12=15-17, ...)
    state_history.append(np.concatenate((yk, rg, r12, r1g, rg1, rg2), axis=None))

# convert list to numpy array
state_history = np.array(state_history)

In [ ]:
# odeint integration 
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot(y[0:,0], y[0:,1], y[0:,2], c='red')
ax.plot(y[0:,3], y[0:,4], y[0:,5], c='blue')

# Colliding objects

In [ ]:
c= 3e8 #m/sec

def grav_force(mass_1, mass_2, r):
    """
    provides the force as a (x,y,z) array 
    must be given two masses and a (x,y,z) distance array
    """
    return r*(G*mass_1*mass_2)/(np.linalg.norm(r)**3)

def get_r(pos1, pos2):
    """
    provides the (x,y,z) distance array
    requires pos1 and pos2 to be (x,y,z) arrays
    """
    r_vec = np.array([pos1[0]-pos2[0], pos1[1]-pos2[1], pos1[2]-pos2[2]])
    return r_vec

def gwpl(m1, m2, r):
    """Gravitational wave power loss. 
    r must be a (x,y,z) array 
    P is power """
    P = ((G**4)*(m1+m2)*(m1*m2)**2)/((c*np.linalg.norm(r))**5)
    return P

def leapfrogintegrate(acc1, acc2, pos1, pos2, vel1, vel2, mass1, mass2):
    """
    pos1 = [x,y,z] array
    pos2 = [x,y,z] array
    vel1 = [x,y,z] array
    vel2 = [x,y,z] array
    
    does leapfrog integration and returns position and velocity values 
    """
    
    # Create empty arrays 
    x1 = np.zeros(N)
    y1 = np.zeros(N)
    z1 = np.zeros(N)
    x2 = np.zeros(N)
    y2 = np.zeros(N)
    z2 = np.zeros(N)
    vx1 = np.zeros(N)
    vy1 = np.zeros(N)
    vz1 = np.zeros(N)
    vx2 = np.zeros(N)
    vy2 = np.zeros(N)
    vz2 = np.zeros(N)
    
    # initialize 
    for index in range(N-1):
        # set initial step 
        if index == 0:
            x1[index] = pos1[0]
            x2[index] = pos2[0]
            y1[index] = pos1[1]
            y2[index] = pos2[1]
            z1[index] = pos1[2]
            z2[index] = pos2[2]
            vx1[index] = vel1[0]
            vx2[index] = vel2[0]
            vy1[index] = vel1[1]
            vy2[index] = vel2[1]
            vz1[index] = vel1[2]
            vz2[index] = vel2[2]
        # integrate 
        else:
            r = get_r([x1[index-1],y1[index-1],z1[index-1]], [x2[index-1],y2[index-1],z2[index-1]])
            F = grav_force(mass1, mass2, r)
            # sets next velocity. 
            # uses the particle's acceleration, last jump, and grav force
            vx1[index] = vx1[index-1]+dt*(acc1-F[0]/mass1)
            vy1[index] = vy1[index-1]+dt*(acc1-F[1]/mass1)
            vz1[index] = vz1[index-1]+dt*(acc1-F[2]/mass1)
            vx2[index] = vx2[index-1]+dt*(acc2+F[0]/mass2)
            vy2[index] = vy2[index-1]+dt*(acc2+F[1]/mass2)
            vz2[index] = vz2[index-1]+dt*(acc2+F[2]/mass2)
            # sets next position. 
            # uses the last jump and next velocity 
            x1[index] = x1[index-1]+dt*vx1[index]
            x2[index] = x2[index-1]+dt*vx2[index]
            y1[index] = y1[index-1]+dt*vy1[index]
            y2[index] = y2[index-1]+dt*vy2[index]
            z1[index] = z1[index-1]+dt*vz1[index]
            z2[index] = z2[index-1]+dt*vz2[index]
    
    return x1,x2,y1,y2,z1,z2,vx1,vx2,vy1,vy2,vz1,vz2

In [ ]:
solar_mass = 1.98847e30 #kg
mass_obj1 = myNS.get_mass() * solar_mass #1e26
mass_obj2 = myNS2.get_mass() * solar_mass

pos_obj1 = np.array([0, 500, 0])
pos_obj2 = np.array([5045, 0, 909])

v_obj1 = np.array([0, 0, 0])
v_obj2 = np.array([30, 40, 50])

N = 5000
dt = .1

## Orbit decay 
Code showing the decay of NS2 into the orbit of NS1. In this simulation, NS1 is stationary and NS2 is acting like a satellite around it. 

In [ ]:
t = 0
name = 'NS2'
orbit = 50e5 # int(input ("original orbit /m")) 
instantTime = 3600
radius = orbit + myNS.radius 
P = gwpl(mass_obj1,mass_obj2,radius)
velocity = ((G*mass_obj1)/radius)**0.5
totalEnergy = -0.5*(G*mass_obj1*mass_obj2)/radius
kmorbit = float(orbit/1000)
orbitList= [orbit]
timeList = [t]


#--------------------------------------------------------------------------------

while orbit >= minimum:
    workdone = P*instantTime
    totalEnergy = totalEnergy - workdone 
    radius = ((-0.5)*G*mass_obj1*mass_obj2/totalEnergy)
    velocity =((G*mass_obj1)/radius)**0.5 
    orbit = radius - myNS.radius 
    t += instantTime
    orbitList.append(orbit/1000)
    timeList.append(t/31536000)
    if orbit < minimum:    
        break
years = float(t/31536000)
print ("years taken: %s" % (years))

In [ ]:
plt.plot(timeList, orbitList, color = "blue", lw = 2)
plt.plot([0,years],[200,200],color = "red", linestyle = "--", lw =2)
plt.title("Orbital Decay Simulation of " + name, fontsize=18)
plt.xlabel("time/years", fontsize =18)
plt.ylabel("orbit/km", fontsize = 18)
plt.axis([0,years,150,kmorbit])
ax = plt.axes()        
ax.yaxis.grid(True) 
ax.xaxis.grid(True) 

plt.show()

## Gravitational wave orbit decay
This solver takes the approach in http://www.tapir.caltech.edu/~chirata/ph236/2011-12/lec15.pdf page 5. 
That is, we solve for 

\begin{equation}
\frac{dr}{dt} = \frac{-64}{5} \frac{M_{1}M_{2}(M_{1}+M_{2})}{a^{3}}
\end{equation} 

by accounting for the loss of orbital energy from gravitational waves. 

The thought is that if this works, it provides the change of radius. From there, you give them circular orbits? Idk. Then have them inspiral down. 

In [ ]:
a = 0.0# start time
b = 300.0  # end time
N = 1000
h = (b-a)/N

mass_obj1 = 30
mass_obj2 = 60

def r_dot(r,t):
    #drdt = (-65/5)*(mass_obj1*mass_obj2*(mass_obj1+mass_obj2))/(r**3)
    drdt = 1/(r**3)
    return drdt

tpoints = np.arange(a,b,h)
r = np.zeros(N)
r[0] = 45 #np.linalg.norm(get_r(pos_obj1,pos_obj2))

# Loop over points to solve for r
#for i in range(N-1):
#    r[i+1] = r[i] + h*r_dot(r[i], tpoints[i])
    
for i in range(N-1):
    k1 = h*r_dot(r[i], tpoints[i])
    k2 = h*r_dot(r[i]+k1/2, tpoints[i]+h/2)
    k3 = h*r_dot(r[i]+k2/2, tpoints[i]+h/2)
    k4 = h*r_dot(r[i]+k3, tpoints[i]+h)
    r[i+1] = r[i] + (1/6)*(k1+2*k2+2*k3+k4)

actual = (4*tpoints)**(1/4) +r[0]
sol = odeint(r_dot,r[0],tpoints)

plt.plot(tpoints, actual, label='actual', c='yellow')
plt.plot(tpoints, r, label='distance', c='cyan')
plt.plot(tpoints,sol, label='built-in', c='red')
plt.xlabel("t")
plt.ylabel('distance')
plt.legend()
plt.show()

In [ ]:
r